# Setup

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

# Add Chat History

In many Q&A applications, we want to allow the user to have a back-and-forth conversation, meaning the application needs:
- some sort of "memory" of past questions and answers, 
- some logic for incorporating those into its current thinking.

In this guide, we will focus on **adding some logic for incorporating historical messages**.

Further details on chat history management is [**covered here**](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/)

We'll work off the Q&A app we built over the [**LLM Powered Autonomous Agents**](https://lilianweng.github.io/posts/2023-06-23-agent/) blog post by Lilan Weng in the [**Quickstart**](https://python.langchain.com/v0.1/docs/use_cases/question_answering/quickstart/)

We'll need to update two things about our existing app:
1. **Prompt**: Update our prompt to support historical messages as an input.
2. **Contextualizing questions**: Add a sub-chain that:
    - takes the latest user question,
    - reformulates it in the contextof the chat history.
    
    This is needed in the case the latest question references some context from past messages.

    For example, if a user asks a follow-up question like: *"Can you elaborate on the second point?"*, this cannot be understood without the context of the previous message.

    Therefore, we can't effectively perform retrieval with a question like this.

# Chain Without Chat History

Here is the Q&A app we built over the LLM Powered Autonomous Agents blog post by Lilian Weng in the Quickstart:

In [6]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is a technique that breaks down complex tasks into smaller and simpler steps. It allows models to think step by step and transform big tasks into manageable ones. Different methods like Chain of Thought and Tree of Thoughts can be used for task decomposition.'

# Contextualizing the Question

First, we will need to define a sub-chain that:
- takes historical messages and the latest user question,
- reformulates the question if it makes reference to any information in the historical information.

We'll use a prompt that includes a `MessagesPlaceholder` variable under the name "chat_history".

This allows us to pass in a list of Messages to the prompt using "chat_history" input key.

These messages will be inserted:
- after the system message,
- before the human message containing the latest question.

Note that we leverage a helper function `create_history_aware_retriever` for this step, which:
- manages the case where `chat_history` is empty,
- otherwise, applies `prompt | llm | StrOutputParser() | retriever` in sequence.

`create_history_aware_retriever` constructs a chain that:
- accepts keys `input` and `chat_history` as input,
- has the same output schema as a retriever.

In [10]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [12]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as it is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

This chain prepends a rephrasing of the input query to our retriever, so that the retrieval incorporates the context of the conversation.